In [1]:
import scraper_eod as s
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
import time
from scipy.stats import norm
import seaborn as sns
m = s.StockMongo()
import indicator_daily as i

C:\Users\ander\Documents\yahooScraper\scrape\lib\site-packages\cryptography\x509\base.py:531: CryptographyDeprecationWarning: Parsed a negative serial number, which is disallowed by RFC 5280.
  return rust_x509.load_pem_x509_certificate(data)


In [2]:
portfolio = m.get_portfolio(None)

In [3]:
portfolio.count()

C:\Users\ander\AppData\Local\Temp/ipykernel_31740/1202350323.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  portfolio.count()


12

In [4]:
port = pd.DataFrame.from_records(portfolio)

In [5]:
port

,_id,strike_date,rel_risk,high_put_value,low_call_value,high_call_value,low_put_value,highStrikePut,lowStrikeCall,lowStrikePut,highStrikeCall,p,r,ticker,date,value,direction
0,639f90d0271dab5a67b27d79,2023-01-20,0.68200,4.80,11.31,5.80,8.90,87.65,167.7,92.65,172.7,1.088,0.736305,FANG,2022-12-18,10.61,1
1,639f90d0271dab5a67b27d7a,2023-01-20,0.45096,7.20,49.20,21.37,7.00,1600.00,2175.0,1710.00,2300.0,0.408,0.879950,BKNG,2022-12-18,41.80,1
2,639f90d0271dab5a67b27d7b,2023-02-17,0.44000,3.00,4.50,4.30,3.60,105.00,175.0,110.00,180.0,0.408,0.496203,SNOW,2022-12-18,2.10,1
3,639f90d0271dab5a67b27d7c,2023-02-17,0.39200,2.95,3.70,3.95,2.74,155.00,210.0,160.00,215.0,0.408,1.024109,BA,2022-12-18,0.54,1
4,63a2e225fb3bfd92736812de,2023-02-17,0.40000,6.80,8.90,7.85,7.95,230.00,340.0,235.00,345.0,0.408,0.596819,NFLX,2022-12-21,3.25,1
5,63a2e226fb3bfd92736812df,2023-02-17,0.37600,1.32,2.34,1.76,1.84,45.00,75.0,47.50,77.5,0.408,0.364630,SQ,2022-12-21,1.54,1
6,63a2e226fb3bfd92736812e0,2023-02-17,0.36200,3.42,4.22,4.31,3.30,130.00,190.0,135.00,195.0,0.408,0.925014,NVDA,2022-12-21,0.68,1
7,63a2e226fb3bfd92736812e1,2023-02-17,0.35200,0.72,2.03,1.04,1.47,37.50,65.0,40.00,67.5,0.408,0.318059,DOCU,2022-12-21,2.06,1
8,63a2e45e1a9675bf13f1e581,2023-02-17,0.51000,5.15,9.75,7.00,6.50,220.00,380.0,230.00,390.0,0.408,0.773658,ENPH,2022-12-21,5.95,1
9,63a2e45e1a9675bf13f1e582,2023-01-27,0.47200,2.08,4.40,2.08,2.04,160.00,240.0,160.00,245.0,0.408,0.439498,MRNA,2022-12-21,2.28,1


In [6]:
port

,_id,strike_date,rel_risk,high_put_value,low_call_value,high_call_value,low_put_value,highStrikePut,lowStrikeCall,lowStrikePut,highStrikeCall,p,r,ticker,date,value,direction
0,639f90d0271dab5a67b27d79,2023-01-20,0.68200,4.80,11.31,5.80,8.90,87.65,167.7,92.65,172.7,1.088,0.736305,FANG,2022-12-18,10.61,1
1,639f90d0271dab5a67b27d7a,2023-01-20,0.45096,7.20,49.20,21.37,7.00,1600.00,2175.0,1710.00,2300.0,0.408,0.879950,BKNG,2022-12-18,41.80,1
2,639f90d0271dab5a67b27d7b,2023-02-17,0.44000,3.00,4.50,4.30,3.60,105.00,175.0,110.00,180.0,0.408,0.496203,SNOW,2022-12-18,2.10,1
3,639f90d0271dab5a67b27d7c,2023-02-17,0.39200,2.95,3.70,3.95,2.74,155.00,210.0,160.00,215.0,0.408,1.024109,BA,2022-12-18,0.54,1


In [7]:
formula = list(m.stock_data.options_analisys.find({'ticker': port.iloc[0]['ticker']}))

In [8]:
#apple = m.get_options(port.iloc[0]['ticker'])

symbols = m.stock_data.options_data4.find({'sym': port.iloc[0]['ticker']})
cleanSymbols = [pd.DataFrame.from_records(s['options']) for s in symbols]
op = pd.concat(cleanSymbols)
op.expirationDate = pd.to_datetime(op.expirationDate, format='%Y-%m-%d')
op = op[op['lastTradeDateTime']!='0000-00-00 00:00:00']
op.lastTradeDateTime = pd.to_datetime(op.lastTradeDateTime, format='%Y-%m-%d').dt.date
op.updatedAt = pd.to_datetime(op.updatedAt, format='%Y-%m-%d').dt.date
op.date = op.date - datetime.timedelta(days=1)
#op = op.set_index('date')
o = op

In [9]:
#o = o.reset_index()
call1 = o[(o['strike'] == port.iloc[0]['lowStrikeCall'])&(o['expirationDate'] == port.iloc[0]['strike_date'])&(o['type'] == 'CALL')][['date', 'lastPrice', 'expirationDate', 'strike', 'daysBeforeExpiration', 'iv']]
call1.columns = ['date', 'callLow', 'expirationDate', 'strikeCallLow', 'daysBeforeExpiration', 'iv']
call2 = o[(o['strike'] == port.iloc[0]['highStrikeCall'])&(o['expirationDate'] == port.iloc[0]['strike_date'])&(o['type'] == 'CALL')][['date', 'lastPrice', 'strike']]
call2.columns = ['date', 'callHigh', 'strikeCallHigh']
put1 = o[(o['strike'] == port.iloc[0]['lowStrikePut'])&(o['expirationDate'] == port.iloc[0]['strike_date'])&(o['type'] == 'CALL')][['date', 'lastPrice', 'strike']]
put1.columns = ['date', 'putLow', 'strikePutLow']
put2 = o[(o['strike'] == port.iloc[0]['highStrikePut'])&(o['expirationDate'] == port.iloc[0]['strike_date'])&(o['type'] == 'CALL')][['date', 'lastPrice', 'strike']]
put2.columns = ['date', 'putHigh', 'strikePutHigh']

condor = call1.merge(call2, on='date', how='inner')
condor = condor.merge(put1, on='date', how='inner')
condor = condor.merge(put2, on='date', how='inner')

condor['condor'] = condor.callLow-condor.callHigh-condor.putHigh+condor.putLow

In [14]:
condor

,date,callLow,expirationDate,strikeCallLow,daysBeforeExpiration,iv,callHigh,strikeCallHigh,putLow,strikePutLow,putHigh,strikePutHigh,condor
0,2022-10-18,7.30,2023-01-20,167.7,93,48.5266,4.7,172.7,32.0,92.65,42.9,87.65,-8.30
1,2022-10-19,7.30,2023-01-20,167.7,92,47.9921,4.7,172.7,32.0,92.65,42.9,87.65,-8.30
2,2022-10-20,7.30,2023-01-20,167.7,91,46.9857,5.4,172.7,32.0,92.65,42.9,87.65,-9.00
3,2022-10-21,8.21,2023-01-20,167.7,90,45.4652,5.4,172.7,32.0,92.65,42.9,87.65,-8.09
4,2022-10-22,8.21,2023-01-20,167.7,89,45.4652,5.4,172.7,32.0,92.65,42.9,87.65,-8.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2022-12-14,11.31,2023-01-20,167.7,36,30.1418,8.9,172.7,32.0,92.65,42.9,87.65,-8.49
58,2022-12-15,11.31,2023-01-20,167.7,35,31.8363,8.9,172.7,32.0,92.65,42.9,87.65,-8.49
59,2022-12-16,11.31,2023-01-20,167.7,34,30.5226,8.9,172.7,32.0,92.65,42.9,87.65,-8.49
60,2022-12-17,11.31,2023-01-20,167.7,33,30.6101,8.9,172.7,32.0,92.65,42.9,87.65,-8.49


In [18]:
data_dict = {}
data_dict['high_put_value'] = condor.iloc[0]['putHigh']
data_dict

{'high_put_value': 42.9}

In [11]:
for i, row in port.iterrows():
    test(row)

167.7
2175.0
175.0
210.0


In [13]:
row

date                    2022-10-18 00:00:00
callLow                                 7.3
expirationDate          2023-01-20 00:00:00
strikeCallLow                         167.7
daysBeforeExpiration                     93
iv                                  48.5266
callHigh                                4.7
strikeCallHigh                        172.7
putLow                                 32.0
strikePutLow                          92.65
putHigh                                42.9
strikePutHigh                         87.65
condor                                 -8.3
Name: 0, dtype: object